<div class="align-center">
<a href="https://oumi.ai/"><img src="https://oumi.ai/docs/en/latest/_static/logo/header_logo.png" height="200"></a>

[![Documentation](https://img.shields.io/badge/Documentation-latest-blue.svg)](https://oumi.ai/docs/en/latest/index.html)
[![Discord](https://img.shields.io/discord/1286348126797430814?label=Discord)](https://discord.gg/oumi)
[![GitHub Repo stars](https://img.shields.io/github/stars/oumi-ai/oumi)](https://github.com/oumi-ai/oumi)
<a target="_blank" href="https://colab.research.google.com/github/oumi-ai/oumi/blob/main/notebooks/Oumi - Finetuning Tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</div>

👋 Welcome to Open Universal Machine Intelligence (Oumi)!

🚀 Oumi is a fully open-source platform that streamlines the entire lifecycle of foundation models - from [data preparation](https://oumi.ai/docs/en/latest/resources/datasets/datasets.html) and [training](https://oumi.ai/docs/en/latest/user_guides/train/train.html) to [evaluation](https://oumi.ai/docs/en/latest/user_guides/evaluate/evaluate.html) and [deployment](https://oumi.ai/docs/en/latest/user_guides/launch/launch.html). Whether you're developing on a laptop, launching large scale experiments on a cluster, or deploying models in production, Oumi provides the tools and workflows you need.

🤝 Make sure to join our [Discord community](https://discord.gg/oumi) to get help, share your experiences, and contribute to the project! If you are interested in joining one of the community's open-science efforts, check out our [open collaboration](https://oumi.ai/community) page.

⭐ If you like Oumi and you would like to support it, please give it a star on [GitHub](https://github.com/oumi-ai/oumi).

# Finetuning Overview

In this tutorial, we'll LoRA tune a large language model to produce "thoughts" before producing its output.

We'll use the Oumi framework to streamline the process and achieve high-quality results.

We'll cover the following topics:
1. Prerequisites
2. Data Preparation & Sanity Checks
3. Training Config Preparation
4. Launching Training
5. Monitoring Progress
6. Evaluation
7. Analyzing Results
8. Inference


## Prerequisites

❗**NOTICE:** We recommend running this notebook on a GPU. If running on Google Colab, you can use the free T4 GPU runtime (Colab Menu: `Runtime` -> `Change runtime type`). On Colab, we recommend replacing `HuggingFaceTB/SmolLM2-1.7B-Instruct` with a smaller model like `HuggingFaceTB/SmolLM2-135M-Instruct`, since the T4 only has 16GB VRAM; you can use `Edit -> Find and replace` in the menu bar to do so.

First, let's install Oumi. You can find more detailed instructions [here](https://oumi.ai/docs/en/latest/get_started/installation.html). Here, we include Oumi's GPU dependencies.

In [1]:
# %pip install oumi[gpu]

## Creating our working directory
For our experiments, we'll use the following folder to save the model, training artifacts, and our working configs.

In [1]:
from pathlib import Path

tutorial_dir = "finetuning_tutorial"

Path(tutorial_dir).mkdir(parents=True, exist_ok=True)

## Setup the environment

You may need to set the following environment variables:
- [Optional] HF_TOKEN: Your [HuggingFace](https://huggingface.co/docs/hub/en/security-tokens) token, in case you want to access a private model like Llama.
- [Optional] WANDB_API_KEY: Your [wandb](https://wandb.ai) token, in case you want to log your experiments to wandb.

In [2]:
from dotenv import load_dotenv

load_dotenv()

False

# Getting Started


## Data Preparation
Let's start by checking out our datasets, and seeing what the data looks like. The OpenO1-SFT dataset includes a variety of tasks, including code generation and explanation, with most examples having a "thought" produced prior to the output.

In [3]:
import os
from pathlib import Path

os.environ["HF_HOME"] = Path.expanduser("~/.cache/huggingface")
# os.environ["OUMI_SLURM_CONNECTIONS"] = "shanghong@192.222.48.208"

In [4]:
from oumi.builders import build_tokenizer
from oumi.core.configs import ModelParams
from oumi.datasets import PromptResponseDataset

# Initialize the dataset
tokenizer = build_tokenizer(
    ModelParams(model_name="HuggingFaceTB/SmolLM2-135M-Instruct")
)
dataset = PromptResponseDataset(
    tokenizer=tokenizer,
    hf_dataset_path="O1-OPEN/OpenO1-SFT",
    prompt_column="instruction",
    response_column="output",
)

# Print a few examples
for i in range(3):
    conversation = dataset.conversation(i)
    print(f"Example {i + 1}:")
    for message in conversation.messages:
        print(f"{message.role}: {message.content[:100]}...")  # Truncate for brevity
    print("\n")

[2025-06-27 00:16:15,429][oumi][rank0][pid:1073141][MainThread][INFO]][models.py:506] Using the model's built-in chat template for model 'HuggingFaceTB/SmolLM2-135M-Instruct'.
[2025-06-27 00:16:15,430][oumi][rank0][pid:1073141][MainThread][INFO]][base_map_dataset.py:91] Creating map dataset (type: PromptResponseDataset)... dataset_name: 'O1-OPEN/OpenO1-SFT'
[2025-06-27 00:16:15,941][oumi][rank0][pid:1073141][MainThread][INFO]][base_map_dataset.py:487] Dataset Info:
	Split: train
	Version: 0.0.0
	Dataset size: 372897013
	Download size: 383545217
	Size: 756442230 bytes
	Rows: 77685
	Columns: ['instruction', 'output']
[2025-06-27 00:16:16,549][oumi][rank0][pid:1073141][MainThread][INFO]][base_map_dataset.py:426] Loaded DataFrame with shape: (77685, 2). Columns:
instruction    object
output         object
dtype: object
Example 1:
user: Consider a regular octagon. How many different triangles can be formed if the octagon is placed insi...
assistant: <Thought>
Alright, I need to figure out h

## Model Preparation

For code generation, we want a model with strong general language understanding and coding capabilities. 

We also want a model that is small enough to train and run on a single GPU.

Some good options include:
- ["microsoft/Phi-3-mini-128k-instruct"](https://huggingface.co/microsoft/Phi-3-mini-128k-instruct)
- ["google/gemma-2b"](https://huggingface.co/google/gemma-2b)
- ["Qwen/Qwen2-1.5B-Instruct"](https://huggingface.co/Qwen/Qwen2-1.5B-Instruct)
- ["meta-llama/Llama-3.2-3B-Instruct"](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)
- ["HuggingFaceTB/SmolLM2-1.7B-Instruct"](https://huggingface.co/HuggingFaceTB/SmolLM2-1.7B-Instruct)


For this tutorial, we'll use "HuggingFaceTB/SmolLM2-1.7B-Instruct" as our base model.



## Initial Model Responses

Let's see how our model performs on an example prompt.

In [5]:
%%writefile $tutorial_dir/infer.yaml

model:
  model_name: "HuggingFaceTB/SmolLM2-135M-Instruct"
  trust_remote_code: true
  torch_dtype_str: "bfloat16"

generation:
  max_new_tokens: 128
  batch_size: 1

Overwriting finetuning_tutorial/infer.yaml


In [6]:
from oumi.core.configs import InferenceConfig
from oumi.infer import infer

config = InferenceConfig.from_yaml(str(Path(tutorial_dir) / "infer.yaml"))

input_text = (
    "Write a Python function to implement the quicksort algorithm. "
    "Please include comments explaining each step."
)

results = infer(config=config, inputs=[input_text])

print(results[0])

INFO 06-27 00:16:16 [__init__.py:239] Automatically detected platform cuda.
[2025-06-27 00:16:17,395][oumi][rank0][pid:1073141][MainThread][WARNING]][infer.py:33] No inference engine specified. Using the default 'native' engine.
[2025-06-27 00:16:17,396][oumi][rank0][pid:1073141][MainThread][INFO]][models.py:228] Building model using device_map: auto (DeviceRankInfo(world_size=1, rank=0, local_world_size=1, local_rank=0))...
[2025-06-27 00:16:17,425][oumi][rank0][pid:1073141][MainThread][INFO]][models.py:300] Using model class: <class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'> to instantiate model.
[2025-06-27 00:16:18,054][oumi][rank0][pid:1073141][MainThread][INFO]][models.py:506] Using the model's built-in chat template for model 'HuggingFaceTB/SmolLM2-135M-Instruct'.
[2025-06-27 00:16:18,061][oumi][rank0][pid:1073141][MainThread][INFO]][native_text_inference_engine.py:151] Setting EOS token id to `2`
conversation_id='4db64724-2cb8-5c32-afe3-d81708f55a8f' message

## Preparing our training experiment



Let's create a YAML file for our training config:

In [7]:
%%writefile $tutorial_dir/train.yaml

model:
  model_name: "HuggingFaceTB/SmolLM2-135M-Instruct"
  trust_remote_code: true
  torch_dtype_str: "bfloat16"
  tokenizer_pad_token: "<|endoftext|>"
  device_map: "auto"

data:
  train:
    datasets:
      - dataset_name: "PromptResponseDataset"
        split: "train"
        sample_count: 8000
        dataset_kwargs: {
          "hf_dataset_path": "O1-OPEN/OpenO1-SFT",
          "prompt_column": "instruction",
          "response_column": "output",
          "assistant_only": true,
          "instruction_template": "<|im_start|>user\n",
          "response_template": "<|im_start|>assistant\n",
        }
        shuffle: True
        seed: 42
    # collator_name: "text_with_padding"
    seed: 42

training:
  output_dir: "finetuning_tutorial/output"

  # For a single GPU, the following gives us a batch size of 16
  # If training with multiple GPUs, feel free to reduce gradient_accumulation_steps
  per_device_train_batch_size: 2
  gradient_accumulation_steps: 8
  
  # ***NOTE***
  # We set it to 10 steps to first verify that it works
  # Swap to 1500 steps to get more meaningful results.
  # Note: 1500 steps will take 2-3 hours on a single A100-40GB GPU.
  max_steps: 10
  # max_steps: 1500

  learning_rate: 1e-3
  warmup_ratio: 0.1
  logging_steps: 10
  save_steps: 0
  max_grad_norm: 1
  weight_decay: 0.01

  
  trainer_type: "TRL_SFT"
  optimizer: "adamw_torch_fused"
  enable_gradient_checkpointing: True
  gradient_checkpointing_kwargs:
    use_reentrant: False
  ddp_find_unused_parameters: False
  dataloader_num_workers: "auto"
  dataloader_prefetch_factor: 32
  empty_device_cache_steps: 1
  use_peft: true

peft:
  lora_r: 16
  lora_alpha: 32
  lora_dropout: 0.00
  lora_target_modules:
    - "q_proj"
    - "k_proj"
    - "v_proj"
    - "o_proj"
    - "gate_proj"
    - "up_proj"
    - "down_proj"

Overwriting finetuning_tutorial/train.yaml


## Fine-tuning the model

This will start the fine-tuning process using the Oumi framework. Because we set `max_steps: 5`, this should be very quick. The full fine-tuning process may take a few hours, depending on your GPU.

### SINGLE GPU

In [8]:

!oumi train -c "$tutorial_dir/train.yaml"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



   ____  _    _ __  __ _____
  / __ \| |  | |  \/  |_   _|
 | |  | | |  | | \  / | | |
 | |  | | |  | | |\/| | | |
 | |__| | |__| | |  | |_| |_
  \____/ \____/|_|  |_|_____|

⠧ Loading configuration...0m
[2025-06-27 00:16:27,895][oumi][rank0][pid:1073415][MainThread][INFO]][distributed.py:578] Setting random seed to 42 on rank 0.
[2025-06-27 00:16:32,581][oumi][rank0][pid:1073415][MainThread][INFO]][torch_utils.py:81] Torch version: 2.6.0+cu124. NumPy version: 1.26.4
[2025-06-27 00:16:32,581][oumi][rank0][pid:1073415][MainThread][INFO]][torch_utils.py:89] CUDA version: 12.4 
[2025-06-27 00:16:32,581][oumi][rank0][pid:1073415][MainThread][INFO]][torch_utils.py:92] CuDNN version: 90.8.0
[2025-06-27 00:16:32,582][oumi][rank0][pid:1073415][MainThread][INFO]][torch_utils.py:125] CPU cores: 208 CUDA devices: 1
device(0)='NVIDIA H100 80GB HBM3' Capability: (9, 0) Memory: [Total: 79.19GiB Free: 77.65GiB Allocated: 0.0GiB Cached: 0.0GiB]
[2025-06-27 00:16:32,589][oumi][rank0][pid:1073415][Main

### MULTI-GPU

In [9]:
!oumi distributed torchrun -m oumi train -c "$tutorial_dir/train.yaml"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[2025-06-27 00:16:58,240][oumi][rank0][pid:1073724][MainThread][INFO]][distributed_run.py:308] Running the command: ['oumi', 'train', '-c', 'finetuning_tutorial/train.yaml']

   ____  _    _ __  __ _____
  / __ \| |  | |  \/  |_   _|
 | |  | | |  | | \  / | | |
 | |  | | |  | | |\/| | | |
 | |__| | |__| | |  | |_| |_
  \____/ \____/|_|  |_|_____|

⠧ Loading configuration...0m
[2025-06-27 00:17:05,479][oumi][rank0][pid:1073734][MainThread][INFO]][distributed.py:578] Setting random seed to 42 on rank 0.
INFO:datasets:PyTorch version 2.6.0 available.
[2025-06-27 00:17:10,164][oumi][rank0][pid:1073734][MainThread][INFO]][torch_utils.py:81] Torch version: 2.6.0+cu124. NumPy version: 1.26.4
[2025-06-27 00:17:10,164][oumi][rank0][pid:1073734][MainThread][INFO]][torch_utils.py:89] CUDA version: 12.4 
[2025-06-27 00:17:10,167][oumi][rank0][pid:1073734][MainThread][INFO]][torch_utils.py:92] CuDNN version: 90.8.0
[2025-06-27 00:17:10,167][oumi][rank0][pid:1073734][MainThread][INFO]][torch_utils.p

## Evaluation


As an example, let's create an evaluation configuration file!

**Note:** Since we've finetuned our model to produce thoughts before answering, it's very likely to do worse on most evals out-of-the-box.

Many evals do not allow models to decode and thus don't take advantage of things like inference-time reasoning.

In [10]:
%%writefile $tutorial_dir/eval.yaml

model:
  model_name: "finetuning_tutorial/output"
  torch_dtype_str: "bfloat16"

tasks:
  - evaluation_backend: lm_harness
    task_name: mmlu_college_computer_science

output_dir: "finetuning_tutorial/output/evaluation"
generation:
  batch_size: null # This will let LM HARNESS find the maximum possible batch size.

Overwriting finetuning_tutorial/eval.yaml


In [11]:
!oumi evaluate -c "$tutorial_dir/eval.yaml"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



   ____  _    _ __  __ _____
  / __ \| |  | |  \/  |_   _|
 | |  | | |  | | \  / | | |
 | |  | | |  | | |\/| | | |
 | |__| | |__| | |  | |_| |_
  \____/ \____/|_|  |_|_____|

⠇ Loading configuration...0m
[2025-06-27 00:17:42,116][oumi][rank0][pid:1074030][MainThread][INFO]][model_params.py:248] Found LoRA adapter at finetuning_tutorial/output, setting `adapter_model` to `model_name`.
[2025-06-27 00:17:42,118][oumi][rank0][pid:1074030][MainThread][INFO]][model_params.py:265] Setting `model_name` to HuggingFaceTB/SmolLM2-135M-Instruct found in adapter config.
INFO 06-27 00:17:47 [__init__.py:239] Automatically detected platform cuda.
⠋ Running evaluation...[2025-06-27 00:17:48,726][oumi][rank0][pid:1074030][MainThread][WARNING]][lm_harness.py:311] Since you have GPU support, it is highly recommended that you set the `inference_engine` to `VLLM`, instead of the `NATIVE`, for faster evaluation.
⠼ Running evaluation...[2025-06-27 00:18:05,799][oumi][rank0][pid:1074030][MainThread][INFO]][l

## Use the Fine-tuned Model

Once we're happy with the results, we can serve the fine-tuned model for interactive inference:

In [12]:
%%writefile $tutorial_dir/trained_infer.yaml

model:
  model_name: "HuggingFaceTB/SmolLM2-135M-Instruct"
  adapter_model: "finetuning_tutorial/output"
  trust_remote_code: true
  torch_dtype_str: "bfloat16"

generation:
  max_new_tokens: 2048
  batch_size: 1

Overwriting finetuning_tutorial/trained_infer.yaml


In [13]:
from oumi.core.configs import InferenceConfig
from oumi.infer import infer

config = InferenceConfig.from_yaml(str(Path(tutorial_dir) / "trained_infer.yaml"))

input_text = (
    "Write a Python function to implement the quicksort algorithm. "
    "Please include comments explaining each step."
)

results = infer(config=config, inputs=[input_text])

print(results[0])

[2025-06-27 00:18:17,710][oumi][rank0][pid:1073141][MainThread][WARNING]][infer.py:33] No inference engine specified. Using the default 'native' engine.
[2025-06-27 00:18:17,711][oumi][rank0][pid:1073141][MainThread][INFO]][models.py:228] Building model using device_map: auto (DeviceRankInfo(world_size=1, rank=0, local_world_size=1, local_rank=0))...
[2025-06-27 00:18:17,712][oumi][rank0][pid:1073141][MainThread][INFO]][models.py:300] Using model class: <class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'> to instantiate model.


[2025-06-27 00:18:17,946][oumi][rank0][pid:1073141][MainThread][INFO]][models.py:281] Loading PEFT adapter from: finetuning_tutorial/output ...
[2025-06-27 00:18:18,277][oumi][rank0][pid:1073141][MainThread][INFO]][models.py:506] Using the model's built-in chat template for model 'HuggingFaceTB/SmolLM2-135M-Instruct'.
[2025-06-27 00:18:18,281][oumi][rank0][pid:1073141][MainThread][INFO]][native_text_inference_engine.py:151] Setting EOS token id to `2`
conversation_id='4db64724-2cb8-5c32-afe3-d81708f55a8f' messages=[USER: Write a Python function to implement the quicksort algorithm. Please include comments explaining each step., ASSISTANT: ```python
import math

def quicksort(arr):
    """
    Quicksort algorithm implementation.

    Args:
        arr (list): The input list to be sorted.

    Returns:
        list: The sorted list.

    Raises:
        TypeError: If the input is not a list.
    """
    # Check if the input is a list
    if not isinstance(arr, list):
        raise TypeEr